<a href="https://colab.research.google.com/github/MashuAjmera/Image-Morph/blob/master/morph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import cv2
import sys

In [0]:
# creating class for pixels on image (x and y coordinates)
class pts:
	def __init__(self, x,y):
		self.x=x
		self.y=y

In [0]:
# creating class for triangle (3 points)
class triangle:
	def __init__(self, vertex):
		self.vertex=vertex

In [0]:
# creating class for image (contains image, feature points and triangulation)
class data:
	def __init__(self, img, points, tri ):
		self.img = img
		self.points = points
		self.tri = tri

In [0]:
# taking input from user for feature points in images (only 3 points)
def mouseHandler(event,x,y,flags,param):
  if event==cv2.EVENT_LBUTTONDOWN:
    imgdata = param
    if imgdata.points.size()<3:
      
      #creating a dot for user accessibility (circle radius-3 color-yellow)
      cv2.circle(imgdata.img,(x,y), 3, (0,255,255), 5)
      cv2.imshow("Image",imgdata.img)
      
      #displaying chosen coordinates
      print(x," ",y)		
      
      #storing chosen points
      dot=pts(x,y)		
      imgdata.points.append(dot)

In [0]:
# function to define points in intermediate images (using weighted average)
def fillmorph(pts,p1,p2,alpha):
  i=0
  while i<p1.size():
    xmorphed= (1-alpha)*p1[i].x + alpha*p2[i].x
    ymorphed= (1-alpha)*p1[i].y + alpha*p2[i].y

    dot= pts(xmorphed,ymorphed)
    p3.append(dot)
    i+=1

In [0]:
# function to make a triangle if 3 vertices are given
def makeTriangle(a,  b, c):
	temp=triangle()
	temp.vertex[0]=a
	temp.vertex[1]=b
	temp.vertex[2]=c
	return temp

In [0]:
# sorting function comparator based on y coordinates of two points
def compareY(a,b):
    return a.y<b.y

In [0]:
# function to create triangulation of an image given points on 2d plane (for 3 points only(not using any algorithm))
def triangulation(points, rows, cols):
  temp=triangle()  
  sort(points.begin(),points.end(),compareY)
	
  # Boundary points (Top-Left,Bottom-Left,Bottom-Right,Top-Right)
  TL=pts(0,0)
  BL= pts(rows-1,0)
  BR= pts(rows-1,cols-1)
  TR= pts(0,cols-1)
	
  # 8 triangles based on feature points
  temp.append(makeTriangle(TL,BL,points[0]))
  temp.append(makeTriangle(points[0],BL,points[1]))
  temp.append(makeTriangle(points[1],BL,BR))
  temp.append(makeTriangle(points[1],BR,points[2]))
  temp.append(makeTriangle(points[2],BR,TR))
  temp.append(makeTriangle(TR,TL,points[2]))
  temp.append(makeTriangle(points[2],TL,points[0]))
  temp.append(makeTriangle(points[0],points[1],points[2]))
	
  return temp

In [0]:
# creating affine matrix for transformation given 3 points (this function returns inverse of the actual matrix for reverse mapping)
def getAffine(tri1,tri2):
	# matrix based on triangle1 vertices (3 Points)
	A=[
			[tri1.vertex[0].x,tri1.vertex[0].y,1,0,0,0],
			[0,0,0,tri1.vertex[0].x,tri1.vertex[0].y,1],
			[tri1.vertex[1].x,tri1.vertex[1].y,1,0,0,0],
			[0,0,0,tri1.vertex[1].x,tri1.vertex[1].y,1],
			[tri1.vertex[2].x,tri1.vertex[2].y,1,0,0,0],
			[0,0,0,tri1.vertex[2].x,tri1.vertex[2].y,1]
        ]
		
	# matrix based on triangle2 vertices
	B=[tri2.vertex[0].x,tri2.vertex[0].y,tri2.vertex[1].x,tri2.vertex[1].y,tri2.vertex[2].x,tri2.vertex[2].y]
	
	# variables in transformation matrix
	H= (Mat(6,6,CV_64FC1,A).inv())*Mat(6,1,CV_64FC1,B)
	
	C=[
			[H.at<double>(0),H.at<double>(1),H.at<double>(2)],
			[H.at<double>(3),H.at<double>(4),H.at<double>(5)],
			[0,0,1]
		]
		
	# returning inverse of transformation matrix for reverse mapping
	return Mat(3,3,CV_64FC1,C).inv()

In [0]:
# getting orientation of 3 points (ACW or CW)
def orientation( x1,  y1,  x2, y2, px,  py):
	o= ((x2-x1)*(py-y1))-((px-x1)*(y2-y1))
	return 1 if o>0 else -1 if o<0 else 0

In [0]:
# finding if a point lies inside or outside a triangle (using orientation)
def isinTri(t,  p):
	o1= orientation(t.vertex[0].x,t.vertex[0].y,t.vertex[1].x,t.vertex[1].y,p.x,p.y)
	o2= orientation(t.vertex[1].x,t.vertex[1].y,t.vertex[2].x,t.vertex[2].y,p.x,p.y)
	o3= orientation(t.vertex[2].x,t.vertex[2].y,t.vertex[0].x,t.vertex[0].y,p.x,p.y)
	
	return (o1==o2 and o2==o3)

In [0]:
# finding suitable triangle for a point
def findTriangle(t,  p):
	for i in range(len(t)):
		if(isinTri(t[i],p)):
			return i
	return -1

In [0]:
# creating intermediate images based on different values of alpha
def morphedImage(alpha,srcdata,destdata):
  morphdata=data()
		
  # initialising intermediate image
  morphdata.img=Mat(destdata.img.rows,destdata.img.cols,CV_64FC3,Scalar(0,0,0))
		
  # feature points based on weighted average
  fillmorph(morphdata.points,srcdata.points,destdata.points,alpha)
		
  # triangulating above computed points
  morphdata.tri=triangulation(morphdata.points,morphdata.img.rows,morphdata.img.cols)
    	
  #vector<Mat> srcaffine
  #vector<Mat> destaffine
    	
  # getting transformation matrix for each corresponding triangle
  i=0
  while(i<morphdata.tri.size()):
    srcaffine.append(getAffine(srcdata.tri[i],morphdata.tri[i]))
    destaffine.append(getAffine(destdata.tri[i],morphdata.tri[i]))
    i+=1
  i=0    
  while(i<morphdata.img.rows):
    j=0
    while(j<morphdata.img.cols):
      dot=pts(i,j)
      j+=1
			# finding triangle for point
      k= findTriangle(morphdata.tri,dot)
				
      # if no triangle found skip the point
      if(k==-1):
        continue
				
      # finding corresponding points in source and destination image (reverse mapping)
      orig=[i,j,1]
      modified= Mat(3,1,CV_64FC1,orig)
      srcpoints=srcaffine[k]*modified
      destpoints=destaffine[k]*modified
    			
      # typecasting double points to int
      srcx= round(srcpoints.at<double>(0))
      srcy= round(srcpoints.at<double>(1))
      destx= round(destpoints.at<double>(0))
      desty= round(destpoints.at<double>(1))

      # using color interpolation to assign color values for each pixel
      morphdata.img.at<Vec3d>((int)i,(int)j)= (1-alpha)*(srcdata.img.at<Vec3d>(srcx,srcy)) + alpha*(destdata.img.at<Vec3d>(destx,desty))
      i+=1
    	
  # returning intermediate image
  morphdata.img.convertTo(morphdata.img,CV_8UC3)
  return morphdata.img

SyntaxError: ignored

In [0]:
# int argc, const char** argv
if __name__ == '__main__' :

  filename1 = 'Clinton.jpg'
  filename2="Bush.jpg"

  # reading source image and checking for errors    
  src = cv2.imread(filename1);

  #reading destination image and checking for errors
  dest =cv2.imread(filename2);
	
  srcdata=data()
  destdata=data()
	
  srcdata.img=src.clone()
  destdata.img=dest.clone()
	
  # creating window
  cv2.namedWindow("Image")
	
  # user input from mouse for source image
  cv2.setMouseCallback("Image",mouseHandler,srcdata)
  cv2.imshow("Image",srcdata.img)
  cv2.waitKey(0)
	
  # retrieving original image to undo any modification (dots on images) 
  srcdata.img=src
	
  # coverting image for arithmetics
  srcdata.img.convertTo(srcdata.img,CV_64FC3)
	
  # triangulating source image
  srcdata.tri=triangulation(srcdata.points,srcdata.img.rows,srcdata.img.cols)
	
  # user input from mouse for destination image
  cv2.setMouseCallback("Image",mouseHandler,destdata)
  cv2.imshow("Image",destdata.img)
  cv2.waitKey(0)

  # retrieving original image to undo any modification (dots on images) 
  destdata.img=dest
	
  # coverting image for arithmetics
  destdata.img.convertTo(destdata.img,CV_64FC3)
	
  # triangulating destination image
  destdata.tri=triangulation(destdata.points,destdata.img.rows,destdata.img.cols)
	
  # closing any windows created
  cv2.destroyAllWindows()
	
  # creating and saving intermediate frames
  i=0
  step = 0.01
  alpha=0
  while(alpha<=1):
    name= "morphed" + to_string(i) + ".jpg"
    alpha+=step
    i+=1
    cv2.imwrite(name,morphedImage(alpha,srcdata,destdata))
  return 0


SyntaxError: ignored